In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

c:\Users\yusse\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\yusse\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\yusse\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\yusse\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
dataset = pd.read_csv('./merged_df.csv',encoding="utf8")
dataset = dataset.dropna(axis=0)
dataset = dataset.reindex()

print(len(dataset))
dataset.head()

110


,Unnamed: 0.1,Unnamed: 0,company_name,rating,location,size,industry,street_number,street_name,town,...,"Land- und Forstwirtschaft, Fischerei (A)",Produzierendes Gewerbe (B-F),"Handel, Verkehr, Gast- gewerbe (G-I)",Erbrin- gung von Unternehmensdienstleistungen\n(J-N),Erbringung von öffentl. u. privaten Dienstleistungen (O-U),Industry with most employees,Number of Foreigners per Gemeinde,Foreigners as % of Population,Raw population Gain or Loss,Per 1000 Inhabitants
0,0,0,Kern (Germany),4.0,Saarland,1 to 50 Employees,Enterprise Software & Network Solutions,120,In der Kolling,Bexbach,...,17,1 409,1 472,762,915,"Handel, Verkehr, Gast- gewerbe (G-I)",521,"7,0",41.0,"+ 2,3"
1,1,1,NEUE HALBERG-GUSS,3.4,Saarland,1001 to 5000 Employees,Transportation Equipment Manufacturing,16,Kirchstraße,Saarbrücken,...,107,21 412,20 730,31 533,38 149,Erbringung von öffentl. u. privaten Dienstleis...,18 551,"19,2",-277.0,"- 1,5"
3,3,3,Kärcher,3.4,Saarland,10000+ Employees,Machinery Manufacturing,269,Provinzialstraße,Ensdorf,...,.,200,901,.,390,"Handel, Verkehr, Gast- gewerbe (G-I)",168,"14,0",62.0,"+ 9,6"
4,4,4,MTD Products,3.5,Saarland,5001 to 10000 Employees,Machinery Manufacturing,23,Industriestraße,Saarbrücken,...,107,21 412,20 730,31 533,38 149,Erbringung von öffentl. u. privaten Dienstleis...,18 551,"19,2",-277.0,"- 1,5"
5,5,5,Primark,3.5,Saarland,10000+ Employees,"Department, Clothing & Shoe Stores",81-85,Bahnhofstraße,Saarbrücken,...,107,21 412,20 730,31 533,38 149,Erbringung von öffentl. u. privaten Dienstleis...,18 551,"19,2",-277.0,"- 1,5"


In [14]:
dataset2 = dataset.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
dataset2.columns
dataset2['Municipality'] = dataset2['Gemeinde']
dataset2 = dataset2.drop(['Gemeinde'],axis=1)

dataset2.to_csv('training_df.csv',encoding='utf-8')


In [4]:

n = len(dataset["town"].unique())
print(n)
X = dataset.iloc[:, [3,5,6 ,18,19,21,22,23,24,25,26,27,28,29,30,31]].values
dataset.iloc[:, [3,5,6 ,18,19,21,22,23,24,25,26,27,28,29,30,31]].columns

24


Index(['rating', 'size', 'industry', 'km²', 'Einwohner',
       'Grundstückspreise in € pro m²',
       'Land- und Forstwirtschaft, Fischerei (A)',
       'Produzierendes Gewerbe (B-F)', 'Handel, Verkehr, Gast- gewerbe (G-I)',
       'Erbrin- gung von Unternehmensdienstleistungen\n(J-N)',
       'Erbringung von öffentl. u. privaten Dienstleistungen (O-U)',
       'Industry with most employees', 'Number of Foreigners per Gemeinde',
       'Foreigners as % of Population', 'Raw population Gain or Loss',
       'Per 1000 Inhabitants'],
      dtype='object')

In [5]:
import gower

distance_matrix = gower.gower_matrix(X)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split

y = dataset.iloc[:, [9]].values
X_train, X_test, y_train, y_test = train_test_split( distance_matrix, y, test_size=0.20, random_state=42)

# clf = RandomForestClassifier(max_depth=10, random_state=0)
# clf = SVC(gamma='auto')
# clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
# clf = tree.DecisionTreeClassifier()
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(60, 24), random_state=1)

clf.fit(X_train, y_train)


c:\Users\yusse\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(60, 24), random_state=1,
              solver='lbfgs')

In [9]:
y_pred = clf.predict(X_test)


from pprint import pprint
from sklearn.metrics import classification_report
pprint(classification_report(y_test, y_pred ))


('               precision    recall  f1-score   support\n'
 '\n'
 '    Beckingen       1.00      1.00      1.00         1\n'
 '      Bexbach       0.00      0.00      0.00         1\n'
 '  Blieskastel       0.00      0.00      0.00         1\n'
 '   Heusweiler       0.00      0.00      0.00         0\n'
 '       Lebach       1.00      1.00      1.00         1\n'
 '  Neunkirchen       1.00      1.00      1.00         2\n'
 '   Püttlingen       0.33      1.00      0.50         1\n'
 '  Saarbrücken       1.00      1.00      1.00        12\n'
 'Sankt Ingbert       1.00      1.00      1.00         1\n'
 ' Sankt Wendel       1.00      1.00      1.00         1\n'
 '       Wadern       0.00      0.00      0.00         1\n'
 '\n'
 '     accuracy                           0.86        22\n'
 '    macro avg       0.58      0.64      0.59        22\n'
 ' weighted avg       0.83      0.86      0.84        22\n')


c:\Users\yusse\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\yusse\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\yusse\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 